# LLM Iteration 6

**Aim**

Correct the implementation of the function from iteration 5 and round the output.

**Prompt**

The model output should be rounded to 2 decimal places. Update the model to calculate mu and sigma from the mean and standard deviation in a new function, using the equation:
phi = math.sqrt(v + m^2)
mu = math.log(m^2/phi)
sigma = math.sqrt(math.log(phi^2/m^2))
where v is the standard deviation squared and m is the mean

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

In [1]:
import simpy
import numpy as np
import math

In [2]:
class StrokeWard:
    def __init__(self, env, num_beds):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.length_of_stay = {
            "Stroke": {"mu": 7.4, "sigma": 8.6},
            "TIA": {"mu": 1.8, "sigma": 2.3},
            "Complex Neurological": {"mu": 4.0, "sigma": 5.0},
            "Other": {"mu": 3.8, "sigma": 5.2}
        }

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.patient_type_store = []
        ##################################

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)
            ##################################

            wait_time = round(self.env.now - arrival_time, 2)
            self.wait_times.append(wait_time)
            print(f'Patient {patient.id} ({patient.patient_type}) admitted to '
                  f'the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')
            los = np.random.default_rng(patient.seed).\
                lognormal(self.length_of_stay[patient.patient_type]["mu"],
                          self.length_of_stay[patient.patient_type]["sigma"])
            # Length of stay based on patient type
            yield self.env.timeout(round(los, 2))
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))
            ##################################
            print(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2))) # Added extra )
    return mu, sigma

def stroke_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(1.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Stroke",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def tia_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(9.3)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "TIA",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def complex_neurological_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(3.6)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Complex Neurological",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

def other_arrival(env, stroke_ward, rng):
    while True:
        interarrival_time = rng.exponential(3.2)
        yield env.timeout(interarrival_time)
        patient = Patient(stroke_ward.patient_count, "Other",
                          rng.integers(0, np.iinfo(np.int32).max))
        stroke_ward.patient_count += 1
        env.process(stroke_ward.admit_patient(patient))

In [3]:
env = simpy.Environment()
stroke_ward = StrokeWard(env, num_beds=10)

# Set random seed for reproducibility for each patient type
random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env.process(stroke_arrival(env, stroke_ward, rngs["Stroke"]))
env.process(tia_arrival(env, stroke_ward, rngs["TIA"]))
env.process(complex_neurological_arrival(env, stroke_ward,
                                         rngs["Complex Neurological"]))
env.process(other_arrival(env, stroke_ward, rngs["Other"]))

# Set the run length to 2 months (default)
run_length_days = round(60 * 2)

print("Simulation started...")
env.run(until=run_length_days)
print("Simulation ended.")

# Calculate and print the mean wait time for a bed rounded to two decimal 
# places
mean_wait_time = round(np.mean(stroke_ward.wait_times), 2)
print(f"Mean wait time for a bed: {mean_wait_time} days")

Simulation started...
Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: -0.0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0.0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.33
Patient 2 (TIA) admitted to the stroke ward at day 2.02. Wait time: 0.0 days
Patient 3 (Complex Neurological) admitted to the stroke ward at day 3.56. Wait time: 0.0 days
Patient 4 (Other) admitted to the stroke ward at day 5.03. Wait time: 0.0 days
Patient 5 (Stroke) admitted to the stroke ward at day 5.85. Wait time: -0.0 days
Patient 6 (Stroke) admitted to the stroke ward at day 6.35. Wait time: 0.0 days
Patient 7 (Other) admitted to the stroke ward at day 6.74. Wait time: -0.0 days
Patient 2 (TIA) discharged from the stroke ward at day 7.09
Patient 8 (Complex Neurological) admitted to the stroke ward at day 7.75. Wait time: -0.0 days
Patient 9 (Stroke) admitted to the stroke ward at day 7.94. Wait time: -0.0 days

**Checks**

In [4]:
class TestCase:
    def __init__(self, env, ward, rngs):
        self.env = env
        self.ward = ward
        env.process(stroke_arrival(env, ward, rngs["Stroke"]))
        env.process(tia_arrival(env, ward, rngs["TIA"]))
        env.process(complex_neurological_arrival(env, ward,
                                                 rngs["Complex Neurological"]))
        env.process(other_arrival(env, ward, rngs["Other"]))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)

        print('\nLength of stay:')
        for pat_type, los in zip(ward.patient_type_store, ward.los_store):
            print(f'({pat_type}) {los}')

In [5]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_run_length()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: -0.0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0.0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.33
Patient 2 (TIA) admitted to the stroke ward at day 2.02. Wait time: 0.0 days
Patient 3 (Complex Neurological) admitted to the stroke ward at day 3.56. Wait time: 0.0 days
Patient 4 (Other) admitted to the stroke ward at day 5.03. Wait time: 0.0 days
Patient 5 (Stroke) admitted to the stroke ward at day 5.85. Wait time: -0.0 days
Patient 6 (Stroke) admitted to the stroke ward at day 6.35. Wait time: 0.0 days
Patient 7 (Other) admitted to the stroke ward at day 6.74. Wait time: -0.0 days
Patient 2 (TIA) discharged from the stroke ward at day 7.09
Patient 8 (Complex Neurological) admitted to the stroke ward at day 7.75. Wait time: -0.0 days
Patient 9 (Stroke) admitted to the stroke ward at day 7.94. Wait time: -0.0 days
Patient 10 (Other) ad

In [6]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_bed_hold()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: -0.0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.33
Patient 1 (Stroke) admitted to the stroke ward at day 0.33. Wait time: 0.11 days


In [7]:
# Checks length of stay for patients
NUM_BEDS = 10
RUN_LENGTH = 60

random_seeds = {
    "Stroke": 12345,
    "TIA": 54321,
    "Complex Neurological": 98765,
    "Other": 67890
}

rngs = {patient_type: np.random.default_rng(seed) for patient_type, \
        seed in random_seeds.items()}

env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS)

test = TestCase(env, ward, rngs)
test.test_length_of_stay()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: -0.0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0.0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.33
Patient 2 (TIA) admitted to the stroke ward at day 2.02. Wait time: 0.0 days
Patient 3 (Complex Neurological) admitted to the stroke ward at day 3.56. Wait time: 0.0 days
Patient 4 (Other) admitted to the stroke ward at day 5.03. Wait time: 0.0 days
Patient 5 (Stroke) admitted to the stroke ward at day 5.85. Wait time: -0.0 days
Patient 6 (Stroke) admitted to the stroke ward at day 6.35. Wait time: 0.0 days
Patient 7 (Other) admitted to the stroke ward at day 6.74. Wait time: -0.0 days
Patient 2 (TIA) discharged from the stroke ward at day 7.09
Patient 8 (Complex Neurological) admitted to the stroke ward at day 7.75. Wait time: -0.0 days
Patient 9 (Stroke) admitted to the stroke ward at day 7.94. Wait time: -0.0 days
Patient 10 (Other) ad

**Reflection**

Despite asking for the LLM to fix the bug, it failed to change the length of stay distribution parameters as instructed, and instead only followed the first part of the prompt to round to 2 decimal places. Therefore, the length of stay for the patients are again not correct and there is significant queuing in the model.